# Credit Risk Classification

Credit risk poses a classification problem that’s inherently imbalanced. This is because healthy loans easily outnumber risky loans. In this Application there are various techniques to train and evaluate models with imbalanced classes. I used a test dataset of historical lending activity from a peer-to-peer lending services company to build a model that can identify the creditworthiness of borrowers.

In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(
    Path('./Resources/lending_data.csv')
)

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y_variable = lending_data_df["loan_status"]

# Separate the X variable, the features
X_variable = lending_data_df[["loan_size","interest_rate","borrower_income","debt_to_income","num_of_accounts","derogatory_marks","total_debt"]]

In [4]:
# Review the y variable Series
y_variable.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X_variable.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y_variable.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X_variable, y_variable, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression = LogisticRegression(random_state=1)

# Fit the model using training data
logistic_regression.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
y_prediction = logistic_regression.predict(X_test)

### Step 3: Evaluate the model’s performance

In [10]:
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_test, y_prediction))

0.9520479254722232


In [11]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_prediction))

[[18663   102]
 [   56   563]]


In [12]:
# Print the classification report for the model
print(classification_report_imbalanced(y_test, y_prediction))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.91      1.00      0.95      0.91     18765
          1       0.85      0.91      0.99      0.88      0.95      0.90       619

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model scored 85% or above on all of the metrics with high number of false positives. Potentially due to the low input of bad loan data for training the model.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
resampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = resampler.fit_resample(X_train, y_train)

In [14]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

1    56271
0    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [15]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_regression = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
resampled_regression.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
resampled_prediction = resampled_regression.predict(X_resampled)

### Step 3: Evaluate the model’s performance

In [16]:
# Print the balanced_accuracy score of the model 
print(balanced_accuracy_score(y_resampled, resampled_prediction))

0.9947308560359689


In [17]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_resampled, resampled_prediction))

[[55964   307]
 [  286 55985]]


In [18]:
# Print the classification report for the model
print(classification_report_imbalanced(y_resampled, resampled_prediction))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.99      0.99      0.99      0.99      0.99     56271
          1       0.99      0.99      0.99      0.99      0.99      0.99     56271

avg / total       0.99      0.99      0.99      0.99      0.99      0.99    112542



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model is predicting and clasifiying 